In [1]:
from os import chdir
chdir('/home/jovyan')

In [2]:
from lib import postgres as pg

# 1. Feature engineering

Create new features to be used by the model

In [3]:
check_target = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    *
FROM 
    clean.individual_household_power_consumption
) main
ORDER BY 1 ASC
LIMIT 10;
""")
check_target.head()

,_id,datetime,global_active_power,global_reactive_power,voltage,current,sub_metering_1,sub_metering_2,sub_metering_3
0,1,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,3,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,4,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,5,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


## 1.2. Time features

Power consumption behavior is time dependent. As the objecive is to predict the future step, we are going to use the time information for on step ahead for each step. The following features will be considered:
- is_workday
- is_morning
- is_afternoon
- is_night
- is_dawn

In [16]:
check_target = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    present.*,
    CASE WHEN EXTRACT( DOW FROM (datetime + INTERVAL '1' MINUTE) ) in (0,6) THEN FALSE ELSE TRUE END as is_workday,
    EXTRACT( DOW FROM (datetime + INTERVAL '1' MINUTE) ) + 1 as day_of_week_t1,
    EXTRACT( DOY FROM (datetime + INTERVAL '1' MINUTE) ) as day_of_year_t1,
    EXTRACT( WEEK FROM (datetime + INTERVAL '1' MINUTE) ) as week_of_year_t1,
    EXTRACT( DAY FROM (datetime + INTERVAL '1' MINUTE) ) as day_of_month_t1,
    EXTRACT( HOUR FROM (datetime + INTERVAL '1' MINUTE) ) as hour_of_day_t1
FROM 
    clean.individual_household_power_consumption present    
) main
ORDER BY 1 ASC
LIMIT 10;
""")
check_target.head()

,_id,datetime,global_active_power,global_reactive_power,voltage,current,sub_metering_1,sub_metering_2,sub_metering_3,is_workday,day_of_week_t1,day_of_year_t1,week_of_year_t1,day_of_month_t1,hour_of_day_t1
0,1,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,0.06,1.02,False,7.0,350.0,50.0,16.0,17.0
1,2,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,0.06,0.96,False,7.0,350.0,50.0,16.0,17.0
2,3,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,0.12,1.02,False,7.0,350.0,50.0,16.0,17.0
3,4,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,0.06,1.02,False,7.0,350.0,50.0,16.0,17.0
4,5,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,0.06,1.02,False,7.0,350.0,50.0,16.0,17.0
